### A. **Preço da ureia + commodities correlatas (base central)**

* **World Bank - Commodity Price Data (“Pink Sheet”)**: traz série **mensal** de preços de commodities, incluindo **fertilizantes (ureia)** e também **gás natural, petróleo, grãos, outros fertilizantes**, etc. Ótima para montar features consistentes e alinhadas em frequência.
* (Alternativa) **IMF Primary Commodity Prices**: também tem fertilizantes e séries de referência.

**Por que é “a base”**: com ela você já cobre direto vários itens da lista do cliente: gás, petróleo, grãos, nitrogenados substitutos e até proxies de energia.

---

### B. **Câmbio (para preço local e efeito de importação)**

* **BCB PTAX (API OData)**: cotações diárias (compra/venda) e você agrega para mensal (média/último dia útil).

---

### C. **Fretes / logística (proxy robusta e mensal)**

* **NY Fed - Global Supply Chain Pressure Index (GSCPI)**: índice mensal que incorpora custos de transporte (inclui medidas baseadas em frete marítimo como BDI/Harpex) e variáveis de oferta. Serve como proxy muito boa para “frete marítimo / gargalos”.

---

### D. **Geopolítica (guerras, sanções, tensões, tarifas)**

* **Geopolitical Risk Index (GPR)** (Caldara & Iacoviello): série **mensal** amplamente usada como proxy quantitativa de risco geopolítico (guerras/tensão/sanções).
* (Opcional) **Economic Policy Uncertainty (EPU)** via FRED para “política / tarifas / incerteza macro” (também mensal).

---

### E. **Clima (chuvas / ENSO como proxy global)**

* **NOAA ONI (Oceanic Niño Index)**: série mensal em CSV (ENSO), boa proxy de variações climáticas com impacto em agricultura/demanda logística.
---

### F. **Trade flows (China exportação, Índia import/tenders) - opcional**

* **UN Comtrade / WITS**: dá para extrair exportações/importações de ureia (ex.: China) e usar como feature (volume/valor), mas automatização pode exigir mais “engenharia”.

---


In [ ]:
import os
import re
import json
from io import BytesIO
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# =========================
# Config / utilitários
# =========================

WORLD_BANK_PINK_SHEET_MONTHLY_XLSX = (
    "https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx"
)

NYFED_GSCPI_XLSX = (
    "https://www.newyorkfed.org/medialibrary/research/interactives/gscpi/downloads/gscpi_data.xlsx"
)

NOAA_ONI_CSV = "https://psl.noaa.gov/data/correlation/oni.csv"

GPR_XLS = "https://www.matteoiacoviello.com/gpr_files/data_gpr_export.xls"

# BCB PTAX (OData) - exemplo comum com parâmetros:
# https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2020'&@dataFinalCotacao='12-31-2050'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao
BCB_PTAX_BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"


@dataclass
class SeriesSpec:
    out_name: str
    patterns: List[str]  # regex list to find a column in Pink Sheet


# Tentei deixar genérico o suficiente para sobreviver a pequenas mudanças de header.
PINK_SHEET_SERIES: List[SeriesSpec] = [
    SeriesSpec("urea_usd", [r"\burea\b"]),
    SeriesSpec("natural_gas_usd", [r"natural\s*gas", r"\bng\b"]),
    SeriesSpec("crude_oil_usd", [r"crude.*oil", r"\bbrent\b", r"\bwt[i|l]\b"]),
    SeriesSpec("maize_usd", [r"\bmaize\b", r"\bcorn\b"]),
    SeriesSpec("wheat_usd", [r"\bwheat\b"]),
    SeriesSpec("soybeans_usd", [r"\bsoy\b", r"\bsoybeans?\b"]),
    SeriesSpec("ammonia_usd", [r"\bammonia\b"]),
    SeriesSpec("dap_usd", [r"\bdap\b", r"diammonium\s*phosphate"]),
    SeriesSpec("potassium_usd", [r"\bpotassium\b"]),
]


def _safe_mkdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def _http_get(url: str, timeout: int = 60) -> bytes:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.content


def _to_month_start(dt: pd.Series) -> pd.Series:
    d = pd.to_datetime(dt, errors="coerce")
    return d.dt.to_period("M").dt.to_timestamp(how="start")


def _month_index(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[date_col] = _to_month_start(df[date_col])
    df = df.dropna(subset=[date_col])
    return df.set_index(date_col).sort_index()


def _pick_best_column(columns: List[str], patterns: List[str]) -> Optional[str]:
    cols_norm = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in columns}
    for pat in patterns:
        rx = re.compile(pat, flags=re.IGNORECASE)
        matches = [c for c, cn in cols_norm.items() if rx.search(cn)]
        if len(matches) == 1:
            return matches[0]
        if len(matches) > 1:
            # Heurística: se tiver "urea" e "gulf"/"bulk"/"granular", etc, escolha o mais descritivo
            # Caso não, escolha o primeiro em ordem alfabética (estável).
            matches_sorted = sorted(matches, key=lambda x: (len(str(x)), str(x)))
            return matches_sorted[0]
    return None


# =========================
# Loaders
# =========================

def load_pink_sheet_monthly(selected: List[SeriesSpec]) -> pd.DataFrame:
    content = _http_get(WORLD_BANK_PINK_SHEET_MONTHLY_XLSX)
    xls = pd.ExcelFile(BytesIO(content))

    # Normalmente o primeiro sheet já é o "Monthly Prices", mas deixamos robusto:
    sheet_name = xls.sheet_names[1]
    df_raw = pd.read_excel(xls, sheet_name=sheet_name, engine="openpyxl", skiprows=4, header=[0, 1])

    # Flatten MultiIndex columns: juntar nome e unidade com espaço
    df_raw.columns = [' '.join(col).strip() for col in df_raw.columns.values]

    # Descobrir coluna de data (alguns arquivos usam "date" / "Month" / "Time")
    possible_date_cols = [c for c in df_raw.columns if str(c).strip().lower() in ("date", "time", "month")]
    if not possible_date_cols:
        # fallback: primeira coluna
        date_col = df_raw.columns[0]
    else:
        date_col = possible_date_cols[0]

    df = df_raw.rename(columns={date_col: "date"}).copy()
    df["date"] = pd.to_datetime(df["date"].str.replace('M', ''), format='%Y%m', errors='coerce')

    # Selecionar séries por regex
    picked = {}
    missing = []
    for spec in selected:
        col = _pick_best_column(list(df.columns), spec.patterns)
        if col is None:
            missing.append(spec.out_name)
            continue
        picked[spec.out_name] = col

    if missing:
        print("[AVISO] Algumas séries não foram encontradas no Pink Sheet:", missing)
        # Ajuda a ajustar rapidamente:
        print("[DEBUG] Colunas disponíveis (amostra):", list(df.columns)[:30])

    out = df[["date"] + list(picked.values())].rename(columns=picked)

    # Converter tudo para numérico (algumas colunas podem vir como object)
    for c in out.columns:
        if c != "date":
            out[c] = pd.to_numeric(out[c], errors="coerce")

    return _month_index(out, "date")


def load_bcb_ptax_usdbrl(date_start: str, date_end: str) -> pd.DataFrame:
    # A API usa formato dd-mm-aaaa nas strings do parâmetro
    # Vamos aceitar start/end como "YYYY-MM" ou "YYYY-MM-DD" e converter.
    start_dt = pd.to_datetime(date_start) if len(date_start) > 7 else pd.to_datetime(date_start + "-01")
    end_dt = pd.to_datetime(date_end) if len(date_end) > 7 else (pd.to_datetime(date_end + "-01") + pd.offsets.MonthEnd(0))

    start_str = start_dt.strftime("%m-%d-%Y")  # muitos exemplos aceitam MM-DD-YYYY
    end_str = end_dt.strftime("%m-%d-%Y")

    url = (
        f"{BCB_PTAX_BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
        f"?@dataInicial='{start_str}'&@dataFinalCotacao='{end_str}'&$format=json"
        f"&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
    )

    data = _http_get(url)
    j = json.loads(data.decode("utf-8"))
    values = j.get("value", [])
    df = pd.DataFrame(values)
    if df.empty:
        raise RuntimeError("BCB PTAX: retorno vazio. Verifique janela de datas/URL.")

    df["date"] = pd.to_datetime(df["dataHoraCotacao"], errors="coerce")
    df["usdbrl"] = pd.to_numeric(df["cotacaoVenda"], errors="coerce")

    # Agregar para mensal (média)
    df_m = (
        df.dropna(subset=["date", "usdbrl"])
          .assign(date=_to_month_start(df["date"]))
          .groupby("date", as_index=False)["usdbrl"]
          .mean()
    )
    return _month_index(df_m, "date")


def load_nyfed_gscpi() -> pd.DataFrame:
    content = _http_get(NYFED_GSCPI_XLSX)
    xls = pd.ExcelFile(BytesIO(content))
    # Em geral há duas sheet com a série e coluna "GSCPI"
    sheet = xls.sheet_names[1]
    df = pd.read_excel(xls, sheet_name=sheet)

    # Tentar inferir colunas:
    date_col = _pick_best_column(list(df.columns), [r"date", r"month", r"time"]) or df.columns[0]
    val_col = _pick_best_column(list(df.columns), [r"gscpi"]) or df.columns[1]

    out = df.rename(columns={date_col: "date", val_col: "gscpi"})[["date", "gscpi"]].copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out["gscpi"] = pd.to_numeric(out["gscpi"], errors="coerce")
    return _month_index(out, "date")


def load_noaa_oni() -> pd.DataFrame:
    content = _http_get(NOAA_ONI_CSV)
    df = pd.read_csv(BytesIO(content))
    # Esperado: date, ONI
    date_col = _pick_best_column(list(df.columns), [r"date"]) or df.columns[0]
    val_col = _pick_best_column(list(df.columns), [r"oni"]) or df.columns[1]
    out = df.rename(columns={date_col: "date", val_col: "oni"})[["date", "oni"]].copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out["oni"] = pd.to_numeric(out["oni"], errors="coerce")
    return _month_index(out, "date")


def load_gpr() -> pd.DataFrame:
    content = _http_get(GPR_XLS)

    try:
        df = pd.read_excel(BytesIO(content), engine="xlrd")
    except ImportError as e:
        raise ImportError(
            "Para ler arquivos .xls, instale xlrd: pip install xlrd"
        ) from e
    except ValueError:
        # fallback: tenta sem engine explicita
        df = pd.read_excel(BytesIO(content))

    # Normaliza nomes
    cols_norm = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in df.columns}

    # --- 1) Construir coluna date ---
    # Caso A: tem year e month separados
    year_col = next((c for c, cn in cols_norm.items() if cn in ("year", "yr", "yyyy")), None)
    month_col = next((c for c, cn in cols_norm.items() if cn in ("month", "mo", "mm")), None)

    if year_col is not None and month_col is not None:
        out = df[[year_col, month_col]].copy()
        out["date"] = pd.to_datetime(
            dict(year=pd.to_numeric(out[year_col], errors="coerce"),
                 month=pd.to_numeric(out[month_col], errors="coerce"),
                 day=1),
            errors="coerce",
        )
        df2 = df.copy()
        df2["date"] = out["date"]
    else:
        # Caso B: tem date / time / month como coluna única
        date_col = _pick_best_column(list(df.columns), [r"date", r"time", r"month"]) or df.columns[0]
        df2 = df.rename(columns={date_col: "date"}).copy()
        # Se vier como YYYYMM numérico (ex.: 202112), converte
        if pd.api.types.is_numeric_dtype(df2["date"]):
            yyyymm = pd.to_numeric(df2["date"], errors="coerce")
            year = (yyyymm // 100).astype("Int64")
            month = (yyyymm % 100).astype("Int64")
            df2["date"] = pd.to_datetime(dict(year=year, month=month, day=1), errors="coerce")
        else:
            df2["date"] = pd.to_datetime(df2["date"], errors="coerce")

    # --- 2) Selecionar colunas do GPR ---
    # Ele pode ter GPR agregado e também decomposições (ameaça/ato etc.). Vamos puxar tudo que começar com gpr
    gpr_cols = [c for c, cn in cols_norm.items() if cn == "gpr" or cn.startswith("gpr")]

    if not gpr_cols:
        # fallback: tenta achar pelo regex no nome original
        gpr_cols = [c for c in df2.columns if re.search(r"(^|\W)gpr(\W|$)", str(c), flags=re.IGNORECASE)]

    if not gpr_cols:
        raise RuntimeError("Não encontrei nenhuma coluna 'GPR' no arquivo data_gpr_export.xls.")

    keep = ["date"] + gpr_cols
    out = df2[keep].copy()

    # Numeric
    for c in gpr_cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")

    # Padroniza nomes (opcional)
    rename_map = {}
    for c in gpr_cols:
        cn = cols_norm.get(c, str(c).lower())
        # deixa nomes amigáveis
        rename_map[c] = re.sub(r"[^a-z0-9_]+", "_", cn).strip("_")
    out = out.rename(columns=rename_map)

    return _month_index(out, "date")



# =========================
# Feature engineering / EDA
# =========================

def add_seasonality(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    m = out.index.month.astype(int)
    out["month"] = m
    out["month_sin"] = np.sin(2 * np.pi * m / 12)
    out["month_cos"] = np.cos(2 * np.pi * m / 12)
    return out


def add_lags(df: pd.DataFrame, cols: List[str], lags: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for L in lags:
            new_cols[f"{c}_lag{L}"] = out[c].shift(L)
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def add_rolling(df: pd.DataFrame, cols: List[str], windows: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for w in windows:
            new_cols[f"{c}_ma{w}"] = out[c].rolling(w).mean()
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def lag_correlation_table(df: pd.DataFrame, target: str, features: List[str], lags: List[int]) -> pd.DataFrame:
    rows = []
    y = df[target]
    for f in features:
        if f not in df.columns:
            continue
        for L in lags:
            corr = y.corr(df[f].shift(L))
            rows.append({"feature": f, "lag": L, "corr": corr})
    out = pd.DataFrame(rows).dropna()
    out["abs_corr"] = out["corr"].abs()
    return out.sort_values(["abs_corr"], ascending=False).reset_index(drop=True)


def train_feature_importance_ts(
    df: pd.DataFrame,
    target: str,
    drop_cols: Optional[List[str]] = None,
    n_splits: int = 5,
    random_state: int = 42,
) -> Tuple[pd.DataFrame, Dict[str, float]]:
    drop_cols = drop_cols or []
    data = df.dropna(subset=[target]).copy()
    X = data.drop(columns=[target] + drop_cols, errors="ignore")
    y = data[target].copy()

    # Remover colunas não-numéricas
    X = X.select_dtypes(include=[np.number]).copy()

    # Tirar linhas com NA após lags/rolling
    mask = X.notna().all(axis=1) & y.notna()
    X = X[mask]
    y = y[mask]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = {"mae": [], "rmse": [], "r2": []}

    # Modelo simples e interpretável via importâncias + permutação
    model = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        random_state=random_state,
        n_jobs=-1,
    )

    # Avaliação em folds e treino final no full (para importâncias)
    for tr, te in tscv.split(X):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y.iloc[tr], y.iloc[te]
        model.fit(Xtr, ytr)
        pred = model.predict(Xte)
        fold_metrics["mae"].append(mean_absolute_error(yte, pred))
        fold_metrics["rmse"].append(np.sqrt(mean_squared_error(yte, pred)))
        fold_metrics["r2"].append(r2_score(yte, pred))

    metrics_summary = {
        "mae_mean": float(np.mean(fold_metrics["mae"])),
        "rmse_mean": float(np.mean(fold_metrics["rmse"])),
        "r2_mean": float(np.mean(fold_metrics["r2"])),
        "n_obs": int(len(X)),
        "n_features": int(X.shape[1]),
    }

    # Treino final para importâncias
    model.fit(X, y)

    imp_rf = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

    perm = permutation_importance(model, X, y, n_repeats=15, random_state=random_state, n_jobs=-1)
    imp_perm = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)

    out = pd.DataFrame({
        "feature": X.columns,
        "rf_importance": imp_rf.reindex(X.columns).values,
        "perm_importance": imp_perm.reindex(X.columns).values,
    }).sort_values(["perm_importance", "rf_importance"], ascending=False)

    return out.reset_index(drop=True), metrics_summary


def plot_series(df: pd.DataFrame, cols: List[str], outpath: str, title: str) -> None:
    plt.figure(figsize=(12, 5))
    for c in cols:
        if c in df.columns:
            plt.plot(df.index, df[c], label=c)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath, dpi=140)
    plt.close()


def plot_bar(df: pd.DataFrame, x: str, y: str, outpath: str, title: str, top_n: int = 20) -> None:
    d = df.head(top_n).copy()
    plt.figure(figsize=(10, 6))
    plt.barh(d[x][::-1], d[y][::-1])
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=140)
    plt.close()

In [ ]:
OUTDIR = os.path.abspath("output")
START_DATE = "2000-01"
END_DATE = "2025-12"
TARGET = "Urea  ($/mt)"

In [ ]:
_safe_mkdir(OUTDIR)
figdir = os.path.join(OUTDIR, "figures")
_safe_mkdir(figdir)

# 1) Loaders
print("Baixando Pink Sheet (World Bank)...")
df_prices = load_pink_sheet_monthly(PINK_SHEET_SERIES)

print("Baixando câmbio PTAX (BCB)...")
df_fx = load_bcb_ptax_usdbrl(START_DATE, END_DATE)

print("Baixando GSCPI (NY Fed)...")
df_gscpi = load_nyfed_gscpi()

print("Baixando GPR (Geopolitical Risk)...")
df_gpr = load_gpr()

print("Baixando ONI (NOAA)...")
df_oni = load_noaa_oni()

In [ ]:
df_oni

In [ ]:
# 2) Merge mensal
df = df_prices.join(df_fx, how="outer").join(df_gscpi, how="outer").join(df_gpr, how="outer").join(df_oni, how="outer")
df = df.loc[(df.index >= pd.to_datetime(START_DATE + "-01" if len(START_DATE) == 7 else START_DATE)) &
            (df.index <= pd.to_datetime(END_DATE + "-01" if len(END_DATE) == 7 else END_DATE) + pd.offsets.MonthEnd(0))]

In [ ]:
# Exemplo: preço de ureia em BRL como feature/target alternativo
if "urea_usd" in df.columns and "usdbrl" in df.columns:
    df["urea_brl"] = df["urea_usd"] * df["usdbrl"]

# 3) Feature engineering
df = add_seasonality(df)

base_features = [c for c in df.columns if c not in {TARGET}]
base_features = [c for c in base_features if pd.api.types.is_numeric_dtype(df[c])]

df = add_lags(df, cols=base_features, lags=[1, 2, 3, 6, 12])
df = add_rolling(df, cols=base_features, windows=[3, 6, 12])

In [ ]:
# 4) EDA: correlação com lags (tabela)
feature_cols = [c for c in df.columns if c not in {TARGET}]
lag_table = lag_correlation_table(df, target=TARGET, features=feature_cols, lags=[0, 1, 2, 3, 6, 12])
lag_table.to_csv(os.path.join(OUTDIR, "top_correlacoes.csv"), index=False)

In [ ]:
lag_table

In [ ]:
# 5) Modelo simples p/ ranking de importância (TimeSeriesSplit)
importance_df, metrics = train_feature_importance_ts(
    df=df,
    target=TARGET,
    drop_cols=[],
    n_splits=5,
)
importance_df.to_csv(os.path.join(OUTDIR, "feature_importance.csv"), index=False)

In [ ]:

# 4) EDA: correlação com lags (tabela)
feature_cols = [c for c in df.columns if c not in {TARGET}]
lag_table = lag_correlation_table(df, target=TARGET, features=feature_cols, lags=[0, 1, 2, 3, 6, 12])
lag_table.to_csv(os.path.join(OUTDIR, "top_correlacoes.csv"), index=False)

# 5) Modelo simples p/ ranking de importância (TimeSeriesSplit)
importance_df, metrics = train_feature_importance_ts(
    df=df,
    target=TARGET,
    drop_cols=[],
    n_splits=5,
)
importance_df.to_csv(os.path.join(OUTDIR, "feature_importance.csv"), index=False)

# 6) Salvar dataset final
df_out = df.reset_index().rename(columns={"index": "date"})
df_out.to_csv(os.path.join(OUTDIR, "dataset_mensal.csv"), index=False)

# 7) Gráficos principais
plot_series(df, cols=[TARGET], outpath=os.path.join(figdir, "01_target.png"), title=f"Target: {TARGET}")

plot_series(
    df,
    cols=[c for c in ["natural_gas_usd", "crude_oil_usd", "maize_usd", "wheat_usd", "usdbrl", "gscpi", "gpr", "oni"] if c in df.columns],
    outpath=os.path.join(figdir, "02_principais_drivers.png"),
    title="Drivers (nível) - seleção",
)

plot_bar(
    importance_df,
    x="feature",
    y="perm_importance",
    outpath=os.path.join(figdir, "03_importancia_permutacao.png"),
    title="Importância (Permutation Importance) - Top 20",
    top_n=20,
)

# 8) Log de métricas
with open(os.path.join(OUTDIR, "metrics.txt"), "w", encoding="utf-8") as f:
    for k, v in metrics.items():
        f.write(f"{k}: {v}\n")

print("\nOK! Saídas em:", OUTDIR)
print("Métricas (CV):", metrics)
print("Dica: se quiser prever preço futuro, você pode trocar o target para 'urea_brl' e/ou criar target shift(-h).")